In [4]:
from glob import glob
import subprocess
import numpy as np

In [71]:
CASE = "coupPPE.001"
SIM_DIR = "/glade/u/home/bbuchovecky/projects/cpl_ppe_co2/sims"
ARCH_DIR = "/glade/derecho/scratch/bbuchovecky/archive"
FULL_CASE = f"f.e21.FHIST_BGC.f19_f19_mg17.historical.{CASE}"

h0_excl_set = set(['bc_a1','bc_a4','bc_c1','bc_c4','dst_a1','dst_a2','dst_a3','dst_c1','dst_c2','dst_c3','ncl_a1','ncl_a2','ncl_a3','ncl_c1','ncl_c2','ncl_c3','num_a1','num_a2','num_a3','num_a4','num_c1','num_c2','num_c3','num_c4','pom_a1','pom_a4','pom_c1','pom_c4','so4_a1','so4_a2','so4_a3','so4_c1','so4_c2','so4_c3','soa_a1','soa_a2','soa_c1','soa_c2','ANRAIN','AREI','AREL','AWNC','AWNI','ADRAIN','ADSNOW','ANSNOW','NUMICE','NUMLIQ','NUMRAI','NUMSNO','DTWR_H2O2','DTWR_H2SO4','DTWR_SO2','VQ','UQ','Q850','Q700','Q500','T850','T700','T500','Z850','Z700','Z500','V850','V700','V500','V200','U850','U700','U500','V200'])

h0_deff = glob(f"{ARCH_DIR}/{FULL_CASE}/atm/hist/{FULL_CASE}.cam.h0.*.nc")[0]
ncdump_result = subprocess.run(
    ["ncdump", "-h", h0_deff],
    capture_output=True,
    text=True,
    check=True
)
grep_result = subprocess.run(
    ["grep", "float"],
    input=ncdump_result.stdout,
    capture_output=True,
    text=True,
    check=True
)

h0_defv_dim = dict()
h0_defv_set = set()
for line in grep_result.stdout.strip().split('\n'):
    if line.strip():
        dims = []
        parts = line.strip().split()
        v = parts[1].split("(")[0]
        dims.append(parts[1].split("(")[1])
        for d in parts[2:]:
            if d[-1] == ")":
                dims.append(d[:-1].strip(","))
                break
            else:
                dims.append(d.strip(","))
        h0_defv_dim[v] = tuple(dims)
        h0_defv_set.add(v)

h0_new_set = h0_defv_set - h0_excl_set
h0_new_dim = {k: v for k, v in h0_defv_dim.items() if k in h0_new_set}

In [81]:
print("number of default h0 fields:", len(h0_defv_set))
print("number of excluded h0 fields:", len(h0_excl_set))
print("number of retained h0 fields:", len(h0_new_set))
print("excluded h0 fields not in default:", sorted(h0_excl_set - h0_defv_set))

number of default h0 fields: 356
number of excluded h0 fields: 71
number of retained h0 fields: 310
excluded h0 fields not in default: ['DTWR_H2O2', 'DTWR_H2SO4', 'DTWR_SO2', 'NUMICE', 'NUMLIQ', 'NUMRAI', 'NUMSNO', 'Q500', 'Q700', 'Q850', 'T500', 'T700', 'T850', 'U500', 'U700', 'U850', 'UQ', 'V200', 'V500', 'V700', 'V850', 'Z500', 'Z700', 'Z850', 'num_c4']


In [94]:
print("retained h0 fields with \'lev\' dimension:")
print("".ljust(46, "-"))
n = 0
for v in sorted(h0_new_set):
    if "lev" in h0_defv_dim[v]:
        print(v.ljust(15, " "), h0_defv_dim[v])
        n += 1
print("".ljust(46, "-"))
print("n =", n)

retained h0 fields with 'lev' dimension:
----------------------------------------------
AQRAIN          ('time,', 'lev', 'lat', 'lon')
AQSNOW          ('time,', 'lev', 'lat', 'lon')
CCN3            ('time,', 'lev', 'lat', 'lon')
CLDICE          ('time,', 'lev', 'lat', 'lon')
CLDLIQ          ('time,', 'lev', 'lat', 'lon')
CLOUD           ('time,', 'lev', 'lat', 'lon')
CLOUDFRAC_CLUBB ('time,', 'lev', 'lat', 'lon')
CONCLD          ('time,', 'lev', 'lat', 'lon')
DCQ             ('time,', 'lev', 'lat', 'lon')
DMS             ('time,', 'lev', 'lat', 'lon')
DTCOND          ('time,', 'lev', 'lat', 'lon')
DTV             ('time,', 'lev', 'lat', 'lon')
FICE            ('time,', 'lev', 'lat', 'lon')
FREQI           ('time,', 'lev', 'lat', 'lon')
FREQL           ('time,', 'lev', 'lat', 'lon')
FREQR           ('time,', 'lev', 'lat', 'lon')
FREQS           ('time,', 'lev', 'lat', 'lon')
H2O             ('time,', 'lev', 'lat', 'lon')
H2O2            ('time,', 'lev', 'lat', 'lon')
H2SO4           ('t

In [95]:
print("retained h0 fields")
print("fincl = ", end="")
for v in sorted(h0_new_set):
    print("\'"+v+"\',", end="")

retained h0 fields
fincl = 'AEROD_v','AODABS','AODDUST','AODDUST1','AODDUST2','AODDUST3','AODVIS','AQRAIN','AQSNOW','AQSO4_H2O2','AQSO4_O3','CCN3','CDNUMC','CLDHGH','CLDICE','CLDLIQ','CLDLOW','CLDMED','CLDTOT','CLOUD','CLOUDCOVER_CLUBB','CLOUDFRAC_CLUBB','CONCLD','DCQ','DF_H2O2','DF_H2SO4','DF_SO2','DMS','DMS_SRF','DTCOND','DTV','FICE','FLDS','FLNS','FLNSC','FLNT','FLNTC','FLNTCLR','FLUT','FLUTC','FREQCLR','FREQI','FREQL','FREQR','FREQS','FSDS','FSDSC','FSNS','FSNSC','FSNT','FSNTC','FSNTOA','FSNTOAC','FSUTOA','H2O','H2O2','H2O2_SRF','H2O_CLXF','H2O_CMXF','H2O_SRF','H2SO4','H2SO4_SRF','H2SO4_sfgaex1','ICEFRAC','ICIMR','ICWMR','IWC','LANDFRAC','LHFLX','LWCF','O3','O3colAbove','OCNFRAC','OMEGA','OMEGAT','PBLH','PHIS','PRECC','PRECL','PRECSC','PRECSL','PS','PSL','Q','QFLX','QREFHT','QRL','QRS','QT','RAINQM','RCMINLAYER_CLUBB','RCMTEND_CLUBB','RCM_CLUBB','RELHUM','RELVAR','RHO_CLUBB','RIMTEND_CLUBB','RTP2_CLUBB','RTPTHLP_CLUBB','RVMTEND_CLUBB','SFDMS','SFH2O2','SFH2SO4','SFSO2','SFSOAG','SF